<a href="https://colab.research.google.com/github/sarim711/Dental_OPG_ObjectRecognition/blob/main/Dental_OPG_Object_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Ultralytics YOLOv8
!pip install ultralytics

# Import YOLO
from ultralytics import YOLO
import os

In [ ]:
# Import the zipfile module
import zipfile

# Specify the path to the zip file
zip_path = '/content/Dental OPG (Object Detection).zip'

# Specify the directory where you want to extract the files
extract_path = '/content/'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Unzipping completed!")

In [ ]:
paths = [
    '/content/Dental OPG (Object Detection)/Augmented Dataset/train/images',
    '/content/Dental OPG (Object Detection)/Augmented Dataset/train/labels',
    '/content/Dental OPG (Object Detection)/Augmented Dataset/valid/images',
    '/content/Dental OPG (Object Detection)/Augmented Dataset/valid/labels',
    '/content/Dental OPG (Object Detection)/Augmented Dataset/test/images',
    '/content/Dental OPG (Object Detection)/Augmented Dataset/test/labels'
]

for path in paths:
    if os.path.exists(path):
        print(f"{path}: Exists, contains {len(os.listdir(path))} files")
    else:
        print(f"{path}: Does not exist")

In [ ]:
import yaml

# Define the YAML content
data_yaml = {
    'train': '/content/Dental OPG (Object Detection)/Augmented Dataset/train/images',
    'val': '/content/Dental OPG (Object Detection)/Augmented Dataset/valid/images',
    'test': '/content/Dental OPG (Object Detection)/Augmented Dataset/test/images',
    'nc': 6,
    'names': ['HealthyT', 'C', 'Imp', 'BDR', 'Inf', 'Ft']

}

# Save to data.yaml
with open('/content/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

# Verify the file
with open('/content/data.yaml', 'r') as f:
    print(f.read())


In [ ]:
# Load a pretrained YOLOv8 model
model = YOLO('yolov8l.pt')  # Use 'yolov8s.pt' for better accuracy if needed

# Train the model
model.train(
    data='/content/data.yaml',  # Path to YAML file
    epochs=100,                 # Number of epochs
    batch=16,                   # Batch size (adjust for Colab GPU)
    imgsz=640,                  # Image size
    device='cuda',
    lr0=0.01,
    lrf=0.0001,
    patience=20,                # Early stopping
    project='/content/runs/train',  # Output directory
    name='exp'                  # Experiment name
)

In [ ]:
# Load the trained model
model = YOLO('/content/runs/train/exp/weights/best.pt')

# Evaluate on validation set
metrics = model.val(data='/content/data.yaml')

# Print key metrics
print(f"mAP@50: {metrics.box.map50}")
print(f"mAP@50:95: {metrics.box.map}")

In [ ]:
# Run inference on test set
results = model.predict(
    source='/content/Dental OPG (Object Detection)/Augmented Dataset/test/images',
    save=True,                    # Save predictions
    save_txt=True,                # Save labels in YOLO format
    project='/content/runs/predict',  # Output directory
    name='exp'
)

In [ ]:
def load_labels(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            label = line.strip().split()
            class_id = int(label[0])
            x_center, y_center, width, height = map(float, label[1:])
            labels.append((class_id, x_center, y_center, width, height))
    return labels

# cnvertir les coordonnées normalisées en pixels
def convert_to_pixels(image_width, image_height, labels):
    pixel_labels = []
    for label in labels:
        class_id, x_center, y_center, width, height = label
        x_center_px = x_center * image_width
        y_center_px = y_center * image_height
        width_px = width * image_width
        height_px = height * image_height
        pixel_labels.append((class_id, x_center_px, y_center_px, width_px, height_px))
    return pixel_labels

# Fonction pour afficher l'image avec les box et les classes
def draw_boxes(img, boxes, classes, color=(0, 255, 0), thickness=3, font_scale=1.0, font_thickness=2):
    for box, class_id in zip(boxes, classes):
        x_center, y_center, width, height = box
        x1, y1 = int(x_center - width / 2), int(y_center - height / 2)
        x2, y2 = int(x_center + width / 2), int(y_center + height / 2)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)

        # Ajouter le texte pour la classe avec une taille de police plus grande
        label = f"Class {class_id}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, font_thickness)
    return img

# Dossier des images et labels
test_path = '/content/Dental OPG (Object Detection)/Original Dataset'
label_path = '/content/Dental OPG (Object Detection)/Original Dataset'

# Lister toutes les images dans le répertoire
all_images = [f for f in os.listdir(test_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Sélectionner un nombre d'images aléatoires
random_images = random.sample(all_images, 3)

# Effectuer les prédictions et comparer avec les vrais labels
for image_name in random_images:
    image_path = os.path.join(test_path, image_name)
    label_name = image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
    label_path_full = os.path.join(label_path, label_name)

    # Charger l'image
    img = cv2.imread(image_path)

    # Obtenir les dimensions de l'image pour la conversion des coordonnées
    image_height, image_width, _ = img.shape

    # Charger les labels réels
    true_labels = load_labels(label_path_full)

    # Convertir les labels en coordonnées en pixels
    true_labels_px = convert_to_pixels(image_width, image_height, true_labels)

    # Faire une prédiction sur l'image
    results = model(img)

    # Récupérer les boîtes et les classes prédites
    predicted_boxes = results[0].boxes.xywh.cpu().numpy()
    predicted_classes = results[0].boxes.cls.cpu().numpy()

    # Dessiner les boîtes et les classes sur l'image (Prédictions en vert, Vrais labels en rouge)
    img_pred = draw_boxes(img.copy(), predicted_boxes, predicted_classes, color=(0, 255, 0), thickness=3, font_scale=1.2, font_thickness=3)
    img_true = draw_boxes(img.copy(), [box[1:] for box in true_labels_px], [box[0] for box in true_labels_px], color=(0, 0, 255), thickness=3, font_scale=1.2, font_thickness=3)

    # Afficher l'image avec les prédictions
    plt.figure(figsize=(12, 6))

    # Afficher l'image avec les prédictions
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(img_pred, cv2.COLOR_BGR2RGB))
    plt.title(f"Predictions for {image_name}")

    # Afficher l'image avec les vrais labels
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(img_true, cv2.COLOR_BGR2RGB))
    plt.title(f"Ground Truth for {image_name}")

    plt.show()

    # Comparaison des classes
    print(f"Comparing predictions with ground truth for {image_name}")
    print("Predictions:")
    print(predicted_classes)
    print("True labels:")
    print([label[0] for label in true_labels_px])

In [ ]:
from google.colab import files
import numpy as np
from PIL import Image
from io import BytesIO

# Load the trained model
model_path = '/content/runs/train/exp/weights/best.pt'
if not os.path.exists(model_path):
    print(f"Model not found at {model_path}. Check the path or train the model.")
else:
    model = YOLO(model_path)

# Upload images
print("Upload images for object detection:")
uploaded = files.upload()

# Process and display each image
for img_name, img_data in uploaded.items():
    img = Image.open(BytesIO(img_data)).convert('RGB')
    img_np = np.array(img)

    # Run inference
    results = model.predict(img_np, save=False, verbose=False)

    # Display image with bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(img_np)
    ax = plt.gca()

    for result in results:
        for box in result.boxes:
            xywh = box.xywh.cpu().numpy()[0]
            x, y, w, h = xywh
            x1 = x - w / 2
            y1 = y - h / 2
            rect = plt.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            class_id = int(box.cls)
            class_name = result.names[class_id]
            confidence = float(box.conf)
            plt.text(x1, y1 - 10, f"{class_name} ({confidence:.2f})",
                     color='r', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    plt.axis('off')
    plt.title(f"Detections for {img_name}")
    plt.show()